In [58]:
import sqlite3
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import numpy as np
import sqlite3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from SQLiteDB import SQLiteDB
# 中文队名映射
team_name_mapping = {
    '老鹰': '亚特兰大老鹰',
    '凯尔特人': '波士顿凯尔特人',
    '篮网': '布鲁克林篮网',
    '黄蜂': '夏洛特黄蜂',
    '公牛': '芝加哥公牛',
    '骑士': '克利夫兰骑士',
    '独行侠': '达拉斯独行侠',
    '掘金': '丹佛掘金',
    '活塞': '底特律活塞',
    '勇士': '金州勇士',
    '火箭': '休斯顿火箭',
    '步行者': '印第安纳步行者',
    '快船': '洛杉矶快船',
    '湖人': '洛杉矶湖人',
    '灰熊': '孟菲斯灰熊',
    '热火': '迈阿密热火',
    '雄鹿': '密尔沃基雄鹿',
    '森林狼': '明尼苏达森林狼',
    '鹈鹕': '新奥尔良鹈鹕',
    '尼克斯': '纽约尼克斯',
    '雷霆': '俄克拉荷马城雷霆',
    '魔术': '奥兰多魔术',
    '76人': '费城76人',
    '太阳': '菲尼克斯太阳',
    '开拓者': '波特兰开拓者',
    '国王': '萨克拉门托国王',
    '马刺': '圣安东尼奥马刺',
    '猛龙': '多伦多猛龙',
    '爵士': '犹他爵士',
    '奇才': '华盛顿奇才'
}

# 连接 SQLite 数据库
conn = sqlite3.connect('C:\\code\\Project\\nba_springbootbatis\\springbootbatis\\src\\main\\resources\\nba_total.db')
cursor = conn.cursor()

# 查询比赛数据
query = '''
    SELECT leftName, leftGoal, rightName, rightGoal
    FROM nba_schedule
    WHERE leftGoal != 0 AND rightGoal != 0
      AND leftName !='' AND rightName !='' 
      AND leftName != '中国男篮' AND rightName != '中国男篮'
    ORDER BY date DESC
'''
games = cursor.execute(query).fetchall()

for game in games:
    print(game)

('灰熊', 118, '热火', 120)
('快船', 98, '灰熊', 99)
('魔术', 100, '森林狼', 115)
('勇士', 99, '热火', 102)
('76人', 103, '凯尔特人', 98)
('黄蜂', 97, '篮网', 90)
('猛龙', 89, '马刺', 100)
('活塞', 87, '爵士', 97)
('火箭', 95, '开拓者', 105)
('太阳', 77, '国王', 87)
('雄鹿', 79, '奇才', 91)
('步行者', 93, '骑士', 100)
('尼克斯', 90, '老鹰', 82)
('公牛', 81, '湖人', 107)
('鹈鹕', 82, '掘金', 91)
('雷霆', 88, '独行侠', 79)
('雄鹿', 90, '太阳', 115)
('独行侠', 101, '凯尔特人', 90)
('老鹰', 99, '公牛', 103)
('尼克斯', 91, '活塞', 90)
('76人', 96, '马刺', 80)
('猛龙', 73, '热火', 109)
('勇士', 90, '雷霆', 83)
('开拓者', 68, '黄蜂', 84)
('步行者', 71, '掘金', 86)
('灰熊', 88, '鹈鹕', 77)
('奇才', 73, '国王', 69)
('魔术', 100, '篮网', 102)
('湖人', 93, '骑士', 89)
('快船', 105, '爵士', 88)
('森林狼', 93, '火箭', 83)
('热火', 92, '独行侠', 79)
('国王', 105, '尼克斯', 106)
('猛龙', 76, '爵士', 86)
('凯尔特人', 89, '黄蜂', 84)
('灰熊', 104, '魔术', 98)
('太阳', 100, '雷霆', 99)
('湖人', 87, '老鹰', 86)
('勇士', 96, '骑士', 85)
('步行者', 98, '太阳', 94)
('尼克斯', 85, '篮网', 92)
('黄蜂', 80, '掘金', 66)
('公牛', 77, '活塞', 85)
('76人', 92, '森林狼', 90)
('奇才', 80, '开拓者', 82)
('快船', 11

In [114]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import sqlite3

# 数据标准化
scaler = StandardScaler()

# 连接到 SQLite 数据库
db_path = 'C:\\code\\Project\\nba_springbootbatis\\springbootbatis\\src\\main\\resources\\nba_total.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 查询需要的数据
query = '''
    SELECT leftName, leftGoal, rightName, rightGoal
    FROM nba_schedule
    WHERE leftGoal != 0 AND rightGoal != 0
      AND leftName != '' AND rightName != ''
      AND leftName != '中国男篮' AND rightName != '中国男篮'
    ORDER BY date DESC
'''
games = cursor.execute(query).fetchall()

# 准备存放 X 和 y 的列表
X = []
y = []

# 遍历每场比赛，使用映射找到 team_id，并查询 players_info 表中的数据
for game in games:
    left_name, left_goal, right_name, right_goal = game

    # 在查询队伍信息时，使用映射来转换 team_name
    left_full_name = team_name_mapping.get(left_name, left_name)
    right_full_name = team_name_mapping.get(right_name, right_name)

    # 查询左队的 team_id
    cursor.execute("SELECT team_id FROM team_names WHERE team_name = ?", (left_full_name,))
    left_team_id = cursor.fetchone()

    # 查询右队的 team_id
    cursor.execute("SELECT team_id FROM team_names WHERE team_name = ?", (right_full_name,))
    right_team_id = cursor.fetchone()

    # 如果找到 team_id，则查询 players_info 表中的数据
    if left_team_id and right_team_id:
        # 查询左队的球员信息，并将空值替换为 0
        cursor.execute("SELECT * FROM players_info WHERE team_id = ?", (left_team_id[0],))
        left_team_players = cursor.fetchall()
        left_team_players = [[0 if val is None or val == '' else val for val in player] for player in left_team_players]

        # 查询右队的球员信息，并将空值和空字符串替换为 0
        cursor.execute("SELECT * FROM players_info WHERE team_id = ?", (right_team_id[0],))
        right_team_players = cursor.fetchall()
        right_team_players = [[0 if val is None or val == '' else val for val in player] for player in right_team_players]

        # 定义需要转换为浮点数的列索引
        numeric_columns_indices = [
            0,2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
            23, 24, 25, 26, 27, 28, 29
        ]  # 跳过 player_name, team_name 和 team_id

        # 转换左队数值数据
        left_team_data = np.array(left_team_players)[:, numeric_columns_indices]
        
        left_team_data = left_team_data.astype(np.float64)

        # 转换右队数值数据
        right_team_data = np.array(right_team_players)[:, numeric_columns_indices]
        right_team_data = right_team_data.astype(np.float64)

        # 计算左右队的数值特征的均值
        left_team_avg = np.mean(left_team_data, axis=0)
        right_team_avg = np.mean(right_team_data, axis=0)

        # 合并左右队的均值特征数据
        combined_team_data = np.hstack((left_team_avg, right_team_avg))
        X.append(combined_team_data)

        # 将 leftGoal 和 rightGoal 作为 y 的输出
        y.append([left_goal, right_goal])

# 转换为 DataFrame
df_X = pd.DataFrame(X)
df_y = pd.DataFrame(y, columns=['left_goal', 'right_goal'])

# 合并 X 和 y
df_combined = pd.concat([df_X, df_y], axis=1)

# 删除所有全为 None 或 NaN 的列，并将剩余的 NaN 值替换为 0
df_combined_cleaned = df_combined.dropna(axis=1, how='all').fillna(0)

# 打印前几行结果
print(df_combined_cleaned.head())

# 断开数据库连接
conn.close()


       0            1            2         3           4            5  \
0  147.5  2010.724490  2011.680272  1.877551   81.176871  1901.068027   
1  244.0  1998.334702  1999.425051  1.909651   90.642710  2160.683778   
2  142.5  2007.323944  2008.588028  2.154930  101.404930  2383.714789   
3  278.5  1989.010791  1990.370504  2.185252  109.559353  2520.771583   
4  284.5  1994.000000  1995.352113  2.130282  103.237676  2472.250000   

            6            7          8           9  ...        50        51  \
0  292.476190   648.292517  52.414966  150.482993  ...  0.238526  0.668052   
1  355.977413   767.568789  39.330595  111.535934  ...  0.233439  0.641952   
2  373.841549   821.250000  70.070423  198.341549  ...  0.226575  0.670735   
3  444.516187  1002.723022  41.032374  113.388489  ...  0.238526  0.668052   
4  413.158451   923.684859  31.204225   90.705986  ...  0.139231  0.662759   

          52        53        54        55        56        57  left_goal  \
0  17.810727  6

In [115]:
# 对 X 进行标准化
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# 对 y 进行标准化
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

In [117]:
X_scaled

array([[-1.1885205 ,  1.56067339,  1.5382903 , ..., -0.29118723,
         1.31747616,  0.70643394],
       [ 0.6611807 , -0.17065393, -0.20144675, ...,  0.37437706,
         1.76563117,  0.71335261],
       [-1.28435994,  1.085487  ,  1.09931891, ...,  1.0896702 ,
         1.00370076,  1.60097864],
       ...,
       [ 0.73785225, -1.07617975, -1.05560896, ...,  0.3731645 ,
        -0.82023202, -1.15546822],
       [-1.23644022,  0.94786671,  0.97310809, ...,  0.3731645 ,
        -0.82023202, -1.15546822],
       [-1.15018472,  0.932368  ,  0.93343075, ..., -1.38167192,
        -1.20592735, -0.31565431]])

In [99]:
# 转换 X 和 y 为 NumPy 数组
X = np.array(X)
y = np.array(y)

# 对文本特征进行 OneHot 编码
if len(text_data) > 0:
    flat_text_data = np.array([item for sublist in text_data for item in sublist]).reshape(-1, 1)
    onehot_encoded_text = onehot_encoder.fit_transform(flat_text_data)

    # 将 OneHot 编码结果与数值特征合并
    X_combined = np.hstack((X, onehot_encoded_text))

    # 对 X_combined 进行标准化处理
    X_scaled = scaler.fit_transform(X_combined)

    # 输出数据维度信息
    print(f"输入数据 X 的维度: {X_scaled.shape}")
    print(f"输出数据 y 的维度: {y.shape}")
else:
    print("没有可用的文本数据进行 OneHot 编码，检查数据来源。")

# 断开数据库连接
conn.close()


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (86,) + inhomogeneous part.

In [118]:
from sklearn.preprocessing import StandardScaler

# 对 X 进行标准化
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# 对 y 进行标准化
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

# 切分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# 构建神经网络模型
model = Sequential()

# 第一层：输入层到第一个隐藏层，带有 10 个神经元，激活函数为 ReLU
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))

# 第二层：第一个隐藏层到第二个隐藏层，带有 5 个神经元，激活函数为 ReLU
model.add(Dense(5, activation='relu'))

# 第三层：第二个隐藏层到输出层，带有 2 个神经元，用于回归任务（输出为连续值）
model.add(Dense(2))

# 编译模型，使用均方误差 (MSE) 作为损失函数，Adam 作为优化器
model.compile(optimizer='adam', loss='mse')

# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=6, validation_data=(X_test, y_test))

# 进行预测
y_pred_scaled = model.predict(X_test)

# 将预测值反标准化，恢复到原始的尺度
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# 输出预测结果
print("预测结果：", y_pred[:5])  # 打印前5个预测结果


Epoch 1/10


c:\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.5638 - val_loss: 1.3327
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3086 - val_loss: 1.2409
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1513 - val_loss: 1.1635
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1358 - val_loss: 1.1114
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0033 - val_loss: 1.0718
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1538 - val_loss: 1.0496
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0279 - val_loss: 1.0302
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8551 - val_loss: 1.0112
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9360 - val_loss: 1.0015
Epoch 10/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9457 - val_loss: 0.9977
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
预测结果： [[81.228065 93.07676 ]
 [88.8723   93.26527 ]
 [87.48611  95.95641 ]
 [95.68552  94.22981 ]
 [85.02244  93.77476 ]]


In [119]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 10)             │           590 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 5)              │            55 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 2)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,973 (7.71 KB)

 Trainable params: 657 (2.57 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,316 (5.14 KB)

In [ ]:
np.round(y_pred)

In [120]:
# 保存模型到文件
model.save('nba_score_prediction_model.h5')
print("模型已保存。")


模型已保存。


In [122]:
from tensorflow.keras.models import load_model

# 读取模型，不进行编译
loaded_model = load_model('nba_score_prediction_model.h5', compile=False)
print("模型已加载。")

# 手动编译模型，指定损失函数为 'mse'
loaded_model.compile(optimizer='adam', loss='mse')

# 使用加载的模型进行预测
y_pred_loaded_model = loaded_model.predict(X_test)

# 将预测值反标准化并取整
y_pred_loaded_rounded = np.round(scaler_y.inverse_transform(y_pred_loaded_model))

# 输出预测结果
print("从加载的模型中获得的预测结果：", y_pred_loaded_rounded[:5])


模型已加载。
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
从加载的模型中获得的预测结果： [[81. 93.]
 [89. 93.]
 [87. 96.]
 [96. 94.]
 [85. 94.]]


In [123]:
import joblib

# 假设 scaler_X 是你要保存的 StandardScaler 对象
joblib.dump(scaler_y, 'scaler_y.pkl')
print("StandardScaler 已保存为 scaler_X.pkl")


StandardScaler 已保存为 scaler_X.pkl
